In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,princeville,45.7668,-61.2985,45.518,94,20,2.06,CA,1697602952
1,1,boa vista,2.8197,-60.6733,78.782,89,20,1.54,BR,1697602952
2,2,port-aux-francais,-49.3500,70.2167,36.788,74,99,6.17,TF,1697602953
3,3,badger,64.8000,-147.5333,33.980,89,100,0.00,US,1697602953
4,4,ocean shores,46.9737,-124.1563,57.020,96,100,3.09,US,1697602785


In [5]:
# Assuming city_data_df is your DataFrame and it has 'Lat', 'Lng', and 'Humidity' columns
map_plot = city_data_df.hvplot.points('Lng', 'Lat', 
                                      geo=True, # This enables geo plotting
                                      tiles='OSM', # This sets the tile source to OpenStreetMap
                                      color='Humidity', # This sets the color to represent humidity
                                      size='Humidity', # This sets the size of points to represent humidity
                                      hover_cols=['City']) # This will show city name when you hover over the points

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [6]:
# Assuming 'temp' is a column in city_data_df representing temperature
city_data_df = city_data_df[(city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 100)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
(city_data_df.head())



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,princeville,45.7668,-61.2985,45.518,94,20,2.06,CA,1697602952
1,1,boa vista,2.8197,-60.6733,78.782,89,20,1.54,BR,1697602952
2,2,port-aux-francais,-49.3500,70.2167,36.788,74,99,6.17,TF,1697602953
3,3,badger,64.8000,-147.5333,33.980,89,100,0.00,US,1697602953
4,4,ocean shores,46.9737,-124.1563,57.020,96,100,3.09,US,1697602785


In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
(hotel_df.head())



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,princeville,45.7668,-61.2985,45.518,94,20,2.06,CA,1697602952,
1,1,boa vista,2.8197,-60.6733,78.782,89,20,1.54,BR,1697602952,
2,2,port-aux-francais,-49.3500,70.2167,36.788,74,99,6.17,TF,1697602953,
3,3,badger,64.8000,-147.5333,33.980,89,100,0.00,US,1697602953,
4,4,ocean shores,46.9737,-124.1563,57.020,96,100,3.09,US,1697602785,


In [ ]:
# Set parameters to search for a hotel
radius = 10000 # Search within a radius of 10,000 meters
params = {
    "categories": "accommodation", # Search for accommodations
    "apiKey": geoapify_key # Use your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}" # Filter by circle with center and radius
    params["bias"] = f"proximity:{lat},{lng}" # Bias results by proximity to center
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df


Starting hotel search
princeville - nearest hotel: No hotel found
boa vista - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
ocean shores - nearest hotel: No hotel found
polyarnyy - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
rozhdestveno - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
dhidhdhoo - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
porto real do colegio - nearest hotel: No hotel found
callao - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
afaahiti - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
chadron - nearest hotel: No hotel found
lamu - nearest hotel: No hotel found
monte caseros - nearest hotel: No hotel found
kerikeri - nearest hotel: No hotel found
altai - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel f

In [ ]:
hover_cols = ['hotel_name', 'country']
city_data_df.hvplot.points('Lng', 'Lat', hover_cols=hover_cols)
